In [1]:
import pandas as pd
import numpy as np

# Capítulo 10 - Agregação de dados e operações em grupos

## 10.1 - Funcionamento do GroupBy

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.522829,-0.168150
1,a,two,0.466176,1.489543
2,b,one,0.879409,-1.047445
3,b,two,1.089472,0.564355
4,a,one,1.582854,0.608526


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a    0.508733
b    0.984440
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one     0.530012
      two     0.466176
b     one     0.879409
      two     1.089472
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,0.530012,0.466176
b,0.879409,1.089472


In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005    0.466176
            2006    0.879409
Ohio        2005    0.283321
            2006    1.582854
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.508733,0.643306
b,0.984440,-0.241545


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.530012  0.220188
     two   0.466176  1.489543
b    one   0.879409 -1.047445
     two   1.089472  0.564355

In [15]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterando por grupos

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.522829 -0.168150
1    a  two  0.466176  1.489543
4    a  one  1.582854  0.608526
b
  key1 key2     data1     data2
2    b  one  0.879409 -1.047445
3    b  two  1.089472  0.564355


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -0.522829 -0.168150
4    a  one  1.582854  0.608526
a two
  key1 key2     data1     data2
1    a  two  0.466176  1.489543
b one
  key1 key2     data1     data2
2    b  one  0.879409 -1.047445
b two
  key1 key2     data1     data2
3    b  two  1.089472  0.564355


In [18]:
pieces = dict(list(df.groupby('key1')))

In [19]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.879409,-1.047445
3,b,two,1.089472,0.564355


In [20]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)

In [22]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.522829 -0.168150
1  0.466176  1.489543
2  0.879409 -1.047445
3  1.089472  0.564355
4  1.582854  0.608526
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecionando uma coluna ou um subconjunto de colunas

In [23]:
df.groupby('key1')['data1']

In [24]:
df['data1'].groupby(df['key1'])

In [25]:
df.groupby('key1')[['data2']]

In [26]:
df[['data2']].groupby(df['key1'])

In [27]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.220188
     two   1.489543
b    one  -1.047445
     two   0.564355

In [28]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [29]:
s_grouped

In [30]:
s_grouped.mean()

key1  key2
a     one     0.220188
      two     1.489543
b     one    -1.047445
      two     0.564355
Name: data2, dtype: float64

### Agrupando com dicionários e Series

In [31]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [32]:
people.iloc[2:3, [1, 2]] = np.nan

In [33]:
people

,a,b,c,d,e
Joe,-0.265645,-1.701518,0.609950,-0.185633,-0.214674
Steve,-1.421680,-0.114363,-0.262276,1.387370,-2.114245
Wes,-0.007360,NaN,NaN,0.107763,-0.680081
Jim,1.686302,0.727299,0.304211,0.985452,-1.397554
Travis,0.803828,0.059645,0.633958,-0.071683,-1.629913


In [34]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red'}

In [35]:
by_column = people.groupby(mapping, axis=1)

In [36]:
by_column.sum()

,blue,red
Joe,0.424317,-2.181838
Steve,1.125094,-3.650288
Wes,0.107763,-0.687441
Jim,1.289663,1.016048
Travis,0.562275,-0.766441


In [37]:
map_series = pd.Series(mapping)

In [38]:
map_series

a     red
b     red
c    blue
d    blue
e     red
dtype: object

In [39]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Agrupando com funções

In [40]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.413297,-0.974219,0.914162,0.907582,-2.292308
5,-1.421680,-0.114363,-0.262276,1.387370,-2.114245
6,0.803828,0.059645,0.633958,-0.071683,-1.629913


In [41]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [42]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.265645 -1.701518  0.609950 -0.185633 -0.680081
  two  1.686302  0.727299  0.304211  0.985452 -1.397554
5 one -1.421680 -0.114363 -0.262276  1.387370 -2.114245
6 two  0.803828  0.059645  0.633958 -0.071683 -1.629913

### Agrupando por níveis de índice

In [43]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

In [44]:
hier_df = pd.DataFrame(np.random.randn(4, 5),
                       columns=columns)

In [45]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.394008 -0.247773  0.025887  0.336995 -0.711317
1      1.577823 -1.314418  0.644306 -0.295552 -1.001633
2     -1.203889 -1.619369 -0.267875  1.430985 -0.289471
3      0.986935  1.203514 -0.362833  0.456398  0.216508

In [46]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 - Agregação de dados

🗒️ Métodos otimizados de groupby - pág 523

In [47]:
df

,key1,key2,data1,data2
0,a,one,-0.522829,-0.168150
1,a,two,0.466176,1.489543
2,b,one,0.879409,-1.047445
3,b,two,1.089472,0.564355
4,a,one,1.582854,0.608526


In [48]:
grouped = df.groupby('key1')

In [49]:
grouped['data1'].quantile(0.9)

key1
a    1.359518
b    1.068465
Name: data1, dtype: float64

In [52]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [53]:
grouped.agg(peak_to_peak)

/tmp/ipykernel_19574/4193407341.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.105682,1.657693
b,0.210062,1.611800


In [54]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.508733  1.053486 -0.522829 -0.028327  0.466176  1.024515   
b      2.0  0.984440  0.148537  0.879409  0.931925  0.984440  1.036956   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.582854   3.0  0.643306  0.829393 -0.168150  0.220188  0.608526   
b     1.089472   2.0 -0.241545  1.139715 -1.047445 -0.644495 -0.241545   

                          
           75%       max  
key1                      
a     1.049034  1.489543  
b     0.161405  0.564355

### Aplicação de função nas colunas e aplicação de várias funções

In [55]:
tips = pd.read_csv('examples/tips.csv')

In [56]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [57]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [59]:
grouped_pct = tips.groupby(['day', 'smoker'])

In [61]:
grouped_pct.agg('mean')

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [62]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

/tmp/ipykernel_19574/540516406.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_pct.agg(['mean', 'std', peak_to_peak])


total_bill                               tip            \
                  mean        std peak_to_peak      mean       std   
day  smoker                                                          
Fri  No      18.420000   5.059282        10.29  2.812500  0.898494   
     Yes     16.813333   9.086388        34.42  2.714000  1.077668   
Sat  No      19.661778   8.939181        41.08  3.102889  1.642088   
     Yes     21.276667  10.069138        47.74  2.875476  1.630580   
Sun  No      20.506667   8.130189        39.40  3.167895  1.224785   
     Yes     24.120000  10.442511        38.10  3.516842  1.261151   
Thur No      17.113111   7.721728        33.68  2.673778  1.282964   
     Yes     19.190588   8.355149        32.77  3.030000  1.113491   

                              size                          tip_pct            \
            peak_to_peak      mean       std peak_to_peak      mean       std   
day  smoker                                                                     
Fri  No             2.00  2.250000  0.500000            1  0.151650  0.028123   
     Yes            3.73  2.066667  0.593617            3  0.174783  0.051293   
Sat  No             8.00  2.555556  0.784960            3  0.158048  0.039767   
     Yes            9.00  2.476190  0.862161            4  0.147906  0.061375   
Sun  No             4.99  2.929825  1.032674            4  0.160113  0.042347   
     Yes            5.00  2.578947  0.901591            3  0.187250  0.154134   
Thur No             5.45  2.488889  1.179796            5  0.160298  0.038774   
     Yes            3.00  2.352941  0.701888            2  0.163863  0.039389   

                          
            peak_to_peak  
day  smoker               
Fri  No         0.067349  
     Yes        0.159925  
Sat  No         0.235193  
     Yes        0.290095  
Sun  No         0.193226  
     Yes        0.644685  
Thur No         0.193350  
     Yes        0.151240

In [63]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

/tmp/ipykernel_19574/345979284.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])


total_bill                  tip                size            \
                   foo        bar       foo       bar       foo       bar   
day  smoker                                                                 
Fri  No      18.420000   5.059282  2.812500  0.898494  2.250000  0.500000   
     Yes     16.813333   9.086388  2.714000  1.077668  2.066667  0.593617   
Sat  No      19.661778   8.939181  3.102889  1.642088  2.555556  0.784960   
     Yes     21.276667  10.069138  2.875476  1.630580  2.476190  0.862161   
Sun  No      20.506667   8.130189  3.167895  1.224785  2.929825  1.032674   
     Yes     24.120000  10.442511  3.516842  1.261151  2.578947  0.901591   
Thur No      17.113111   7.721728  2.673778  1.282964  2.488889  1.179796   
     Yes     19.190588   8.355149  3.030000  1.113491  2.352941  0.701888   

              tip_pct            
                  foo       bar  
day  smoker                      
Fri  No      0.151650  0.028123  
     Yes     0.174783  0.051293  
Sat  No      0.158048  0.039767  
     Yes     0.147906  0.061375  
Sun  No      0.160113  0.042347  
     Yes     0.187250  0.154134  
Thur No      0.160298  0.038774  
     Yes     0.163863  0.039389

In [64]:
functions = ['count', 'mean', 'max']

In [65]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

/tmp/ipykernel_19574/1540724723.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [66]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [67]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [68]:
ftuples = [('Durchshnitt', 'mean'), ('Abweichung', np.var)]

In [69]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

/tmp/ipykernel_19574/514018043.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct             total_bill            
            Durchshnitt Abweichung Durchshnitt  Abweichung
day  smoker                                               
Fri  No        0.151650   0.000791   18.420000   25.596333
     Yes       0.174783   0.002631   16.813333   82.562438
Sat  No        0.158048   0.001581   19.661778   79.908965
     Yes       0.147906   0.003767   21.276667  101.387535
Sun  No        0.160113   0.001793   20.506667   66.099980
     Yes       0.187250   0.023757   24.120000  109.046044
Thur No        0.160298   0.001503   17.113111   59.625081
     Yes       0.163863   0.001551   19.190588   69.808518

In [70]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [71]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Devolvendo dados agregados sem índices de linha

In [72]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 - Método apply: separar-aplicar-combinar genérico